<a href="https://colab.research.google.com/github/cookieukw/IA-Simples/blob/main/IA_Simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### importando as dependências

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


### Montar o google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Exemplo de dados de treinamento (substitua com seus próprios dados)

In [12]:
questions = ["Qual é o seu nome?", "Como você está?", "O que você gosta de fazer?"]
answers = ["Meu nome é ChatGPT.", "Estou bem, obrigado!", "Eu gosto de ajudar as pessoas."]

### Tokenização dos dados


In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
question_seqs = tokenizer.texts_to_sequences(questions)
answer_seqs = tokenizer.texts_to_sequences(answers)

### Padding das sequências para ter o mesmo comprimento

In [14]:
max_seq_length = max(len(seq) for seq in question_seqs + answer_seqs)
question_seqs = pad_sequences(question_seqs, maxlen=max_seq_length, padding='post')
answer_seqs = pad_sequences(answer_seqs, maxlen=max_seq_length, padding='post')


### Conversão das respostas em formato one-hot

In [15]:
vocab_size = len(tokenizer.word_index) + 1
answer_one_hot = []
for seq in answer_seqs:
    one_hot_seq = tf.keras.utils.to_categorical(seq, num_classes=vocab_size)
    answer_one_hot.append(one_hot_seq)

answer_one_hot = np.array(answer_one_hot)


### Construção do modelo

In [16]:
model = keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_seq_length),
    LSTM(128, return_sequences=True),  # Use return_sequences=True para saída em cada etapa de tempo
    Dense(vocab_size, activation='softmax')
])

### Salvar modelo no google drive

In [ ]:
# Substitua 'meu_modelo.h5' pelo nome de arquivo desejado
model.save('/content/drive/My Drive/meu_modelo.h5')


### Compilação do modelo

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Treinamento do modelo

In [ ]:
model.fit(question_seqs, answer_one_hot, epochs=100)

 ### Agora você pode usar o modelo treinado para responder a perguntas

In [19]:
input_question = "Qual é o seu nome?"
input_seq = tokenizer.texts_to_sequences([input_question])
input_seq = pad_sequences(input_seq, maxlen=max_seq_length, padding='post')
output_seq = model.predict(input_seq)

1/1 [==============================] - 1s 625ms/step


### Converter as sequências de saída em palavras

In [20]:
output_words = []
for seq in output_seq[0]:
    word = tokenizer.index_word.get(np.argmax(seq))
    if word:
        output_words.append(word)

### Juntar as palavras em uma frase de resposta

In [22]:
output_answer = " ".join(output_words)
print("Resposta:", output_answer)

Resposta: nome nome é chatgpt
